# Despliegue de Modelos en Producción

## Objetivos
- Implementar un pipeline completo de MLOps
- Crear APIs REST para servir modelos de ML
- Containerizar aplicaciones con Docker
- Implementar monitoreo y logging
- Desplegar en la nube (Heroku/AWS)

## Contexto
En esta práctica, tomaremos uno de los modelos desarrollados en el Bloque 2 y lo desplegaremos en un entorno de producción real. Trabajaremos con datos de rendimiento de jugadores para crear un servicio web que prediga el valor de mercado de un jugador.

## 1. Preparación del Entorno

### Instalación de Dependencias

In [ ]:
# Instalar dependencias necesarias
!pip install fastapi uvicorn pandas scikit-learn joblib pydantic python-multipart
!pip install mlflow docker-py requests streamlit plotly

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import json
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

## 2. Preparación del Modelo

### Cargar y Preparar Datos

In [ ]:
# Simular datos de jugadores para el ejemplo
np.random.seed(42)
n_players = 1000

# Generar datos sintéticos de jugadores
data = {
    'age': np.random.randint(18, 35, n_players),
    'goals_season': np.random.poisson(8, n_players),
    'assists_season': np.random.poisson(5, n_players),
    'minutes_played': np.random.randint(500, 3000, n_players),
    'pass_accuracy': np.random.uniform(0.6, 0.95, n_players),
    'defensive_actions': np.random.poisson(50, n_players),
    'league_level': np.random.choice([1, 2, 3, 4, 5], n_players),  # 1=top league, 5=low league
    'position': np.random.choice(['GK', 'DEF', 'MID', 'FWD'], n_players)
}

df = pd.DataFrame(data)

# Crear variable objetivo (valor de mercado) basada en características
# Fórmula simplificada para generar valores realistas
base_value = 1000000  # 1M base
age_factor = (30 - df['age']) / 10  # Penalizar edad
performance_factor = (df['goals_season'] + df['assists_season']) / 10
league_factor = (6 - df['league_level']) / 2
position_bonus = df['position'].map({'GK': 0.8, 'DEF': 0.9, 'MID': 1.0, 'FWD': 1.2})

df['market_value'] = base_value * (1 + age_factor + performance_factor + league_factor) * position_bonus
df['market_value'] = np.maximum(df['market_value'], 100000)  # Valor mínimo

print("Dataset creado:")
print(df.head())
print(f"\nShape: {df.shape}")
print(f"Valor promedio: €{df['market_value'].mean():,.0f}")

### Entrenar Modelo

In [ ]:
# Preparar datos para el modelo
# One-hot encoding para posición
df_encoded = pd.get_dummies(df, columns=['position'], prefix='pos')

# Características para el modelo
feature_columns = ['age', 'goals_season', 'assists_season', 'minutes_played', 
                   'pass_accuracy', 'defensive_actions', 'league_level',
                   'pos_DEF', 'pos_FWD', 'pos_GK', 'pos_MID']

X = df_encoded[feature_columns]
y = df_encoded['market_value']

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluar modelo
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Métricas del modelo:")
print(f"MSE: {mse:,.0f}")
print(f"R²: {r2:.3f}")
print(f"RMSE: {np.sqrt(mse):,.0f}")

### Guardar Modelo y Metadatos

In [ ]:
# Crear directorio para el modelo
os.makedirs('model_artifacts', exist_ok=True)

# Guardar modelo y scaler
joblib.dump(model, 'model_artifacts/player_value_model.pkl')
joblib.dump(scaler, 'model_artifacts/scaler.pkl')

# Guardar metadatos del modelo
model_metadata = {
    'model_type': 'RandomForestRegressor',
    'features': feature_columns,
    'target': 'market_value',
    'training_date': datetime.now().isoformat(),
    'metrics': {
        'mse': float(mse),
        'r2': float(r2),
        'rmse': float(np.sqrt(mse))
    },
    'training_size': len(X_train),
    'test_size': len(X_test)
}

with open('model_artifacts/metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)

print("Modelo y metadatos guardados en 'model_artifacts/'")

## 3. Creación de la API REST

### Estructura de la API con FastAPI

In [ ]:
# Crear el archivo de la API
api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import pandas as pd
import numpy as np
import json
from datetime import datetime
import logging
from typing import List, Dict, Any

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Crear la aplicación FastAPI
app = FastAPI(
    title="Player Value Prediction API",
    description="API para predecir el valor de mercado de jugadores de fútbol",
    version="1.0.0"
)

# Cargar modelo y scaler al inicio
try:
    model = joblib.load('model_artifacts/player_value_model.pkl')
    scaler = joblib.load('model_artifacts/scaler.pkl')
    
    with open('model_artifacts/metadata.json', 'r') as f:
        model_metadata = json.load(f)
        
    logger.info("Modelo cargado exitosamente")
except Exception as e:
    logger.error(f"Error cargando modelo: {e}")
    raise

# Modelos Pydantic para validación
class PlayerData(BaseModel):
    age: int
    goals_season: int
    assists_season: int
    minutes_played: int
    pass_accuracy: float
    defensive_actions: int
    league_level: int
    position: str
    
    class Config:
        schema_extra = {
            "example": {
                "age": 25,
                "goals_season": 15,
                "assists_season": 8,
                "minutes_played": 2500,
                "pass_accuracy": 0.85,
                "defensive_actions": 60,
                "league_level": 1,
                "position": "FWD"
            }
        }

class PredictionResponse(BaseModel):
    predicted_value: float
    confidence_interval: Dict[str, float]
    model_version: str
    prediction_date: str

# Endpoints
@app.get("/")
async def root():
    return {"message": "Player Value Prediction API", "version": "1.0.0"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}

@app.get("/model-info")
async def get_model_info():
    return model_metadata

@app.post("/predict", response_model=PredictionResponse)
async def predict_player_value(player: PlayerData):
    try:
        # Validar datos de entrada
        if player.position not in ['GK', 'DEF', 'MID', 'FWD']:
            raise HTTPException(status_code=400, detail="Posición inválida")
        
        if not (18 <= player.age <= 40):
            raise HTTPException(status_code=400, detail="Edad debe estar entre 18 y 40")
        
        if not (1 <= player.league_level <= 5):
            raise HTTPException(status_code=400, detail="Nivel de liga debe estar entre 1 y 5")
        
        # Preparar datos para predicción
        data = {
            'age': player.age,
            'goals_season': player.goals_season,
            'assists_season': player.assists_season,
            'minutes_played': player.minutes_played,
            'pass_accuracy': player.pass_accuracy,
            'defensive_actions': player.defensive_actions,
            'league_level': player.league_level,
            'pos_DEF': 1 if player.position == 'DEF' else 0,
            'pos_FWD': 1 if player.position == 'FWD' else 0,
            'pos_GK': 1 if player.position == 'GK' else 0,
            'pos_MID': 1 if player.position == 'MID' else 0
        }
        
        # Crear DataFrame
        df = pd.DataFrame([data])
        
        # Escalar características
        X_scaled = scaler.transform(df[model_metadata['features']])
        
        # Hacer predicción
        prediction = model.predict(X_scaled)[0]
        
        # Calcular intervalo de confianza (aproximado)
        # Para Random Forest, usar la desviación estándar de los árboles
        tree_predictions = [tree.predict(X_scaled)[0] for tree in model.estimators_]
        std_dev = np.std(tree_predictions)
        
        confidence_interval = {
            "lower": float(prediction - 1.96 * std_dev),
            "upper": float(prediction + 1.96 * std_dev)
        }
        
        # Log de la predicción
        logger.info(f"Predicción realizada: {prediction:.2f} para jugador {player.position} de {player.age} años")
        
        return PredictionResponse(
            predicted_value=float(prediction),
            confidence_interval=confidence_interval,
            model_version="1.0.0",
            prediction_date=datetime.now().isoformat()
        )
        
    except Exception as e:
        logger.error(f"Error en predicción: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/predict-batch")
async def predict_batch(players: List[PlayerData]):
    try:
        predictions = []
        for player in players:
            # Reutilizar lógica de predicción individual
            result = await predict_player_value(player)
            predictions.append(result)
        
        return {"predictions": predictions, "count": len(predictions)}
        
    except Exception as e:
        logger.error(f"Error en predicción batch: {e}")
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Guardar el código de la API
with open('api_main.py', 'w') as f:
    f.write(api_code)

print("API creada en 'api_main.py'")

## 4. Dockerfile para Containerización

In [ ]:
# Crear Dockerfile
dockerfile_content = '''
FROM python:3.9-slim

WORKDIR /app

# Copiar requirements
COPY requirements.txt .

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Copiar código de la aplicación
COPY api_main.py .
COPY model_artifacts/ ./model_artifacts/

# Exponer puerto
EXPOSE 8000

# Comando para ejecutar la aplicación
CMD ["uvicorn", "api_main:app", "--host", "0.0.0.0", "--port", "8000"]
'''

# Crear archivo requirements.txt
requirements_content = '''
fastapi==0.68.0
uvicorn==0.15.0
pandas==1.3.3
scikit-learn==1.0.2
joblib==1.0.1
pydantic==1.8.2
numpy==1.21.2
python-multipart==0.0.5
'''

# Guardar archivos
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

print("Dockerfile y requirements.txt creados")

## 5. Testing de la API

In [ ]:
# Crear script de testing
test_script = '''
import requests
import json
import time

# URL base de la API (ajustar según despliegue)
BASE_URL = "http://localhost:8000"

def test_health_check():
    """Test del endpoint de salud"""
    response = requests.get(f"{BASE_URL}/health")
    print(f"Health check: {response.status_code}")
    print(f"Response: {response.json()}")
    return response.status_code == 200

def test_model_info():
    """Test del endpoint de información del modelo"""
    response = requests.get(f"{BASE_URL}/model-info")
    print(f"Model info: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    return response.status_code == 200

def test_prediction():
    """Test del endpoint de predicción"""
    player_data = {
        "age": 25,
        "goals_season": 15,
        "assists_season": 8,
        "minutes_played": 2500,
        "pass_accuracy": 0.85,
        "defensive_actions": 60,
        "league_level": 1,
        "position": "FWD"
    }
    
    response = requests.post(f"{BASE_URL}/predict", json=player_data)
    print(f"Prediction: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"Predicted value: €{result['predicted_value']:,.2f}")
        print(f"Confidence interval: €{result['confidence_interval']['lower']:,.2f} - €{result['confidence_interval']['upper']:,.2f}")
    else:
        print(f"Error: {response.text}")
    return response.status_code == 200

def test_batch_prediction():
    """Test del endpoint de predicción batch"""
    players_data = [
        {
            "age": 25,
            "goals_season": 15,
            "assists_season": 8,
            "minutes_played": 2500,
            "pass_accuracy": 0.85,
            "defensive_actions": 60,
            "league_level": 1,
            "position": "FWD"
        },
        {
            "age": 28,
            "goals_season": 5,
            "assists_season": 15,
            "minutes_played": 2800,
            "pass_accuracy": 0.88,
            "defensive_actions": 45,
            "league_level": 1,
            "position": "MID"
        }
    ]
    
    response = requests.post(f"{BASE_URL}/predict-batch", json=players_data)
    print(f"Batch prediction: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"Predictions count: {result['count']}")
        for i, pred in enumerate(result['predictions']):
            print(f"Player {i+1}: €{pred['predicted_value']:,.2f}")
    else:
        print(f"Error: {response.text}")
    return response.status_code == 200

def run_performance_test(n_requests=10):
    """Test de rendimiento"""
    player_data = {
        "age": 25,
        "goals_season": 15,
        "assists_season": 8,
        "minutes_played": 2500,
        "pass_accuracy": 0.85,
        "defensive_actions": 60,
        "league_level": 1,
        "position": "FWD"
    }
    
    times = []
    for i in range(n_requests):
        start_time = time.time()
        response = requests.post(f"{BASE_URL}/predict", json=player_data)
        end_time = time.time()
        
        if response.status_code == 200:
            times.append(end_time - start_time)
        else:
            print(f"Request {i+1} failed: {response.status_code}")
    
    if times:
        avg_time = sum(times) / len(times)
        print(f"Performance test ({n_requests} requests):")
        print(f"Average response time: {avg_time:.3f}s")
        print(f"Min response time: {min(times):.3f}s")
        print(f"Max response time: {max(times):.3f}s")
        print(f"Success rate: {len(times)}/{n_requests} ({100*len(times)/n_requests:.1f}%)")

if __name__ == "__main__":
    print("=== Testing API ===")
    
    print("\n1. Health Check")
    test_health_check()
    
    print("\n2. Model Info")
    test_model_info()
    
    print("\n3. Single Prediction")
    test_prediction()
    
    print("\n4. Batch Prediction")
    test_batch_prediction()
    
    print("\n5. Performance Test")
    run_performance_test()
    
    print("\n=== Testing Complete ===")
'''

with open('test_api.py', 'w') as f:
    f.write(test_script)

print("Script de testing creado en 'test_api.py'")

## 6. Monitoreo y Logging

In [ ]:
# Crear sistema de monitoreo
monitoring_script = '''
import logging
import json
import time
from datetime import datetime
import psutil
import requests
import sqlite3
from typing import Dict, Any

class APIMonitor:
    def __init__(self, api_url: str = "http://localhost:8000", db_path: str = "monitoring.db"):
        self.api_url = api_url
        self.db_path = db_path
        self.setup_logging()
        self.setup_database()
    
    def setup_logging(self):
        """Configurar logging"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('api_monitor.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def setup_database(self):
        """Configurar base de datos para métricas"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Tabla para métricas de sistema
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS system_metrics (
                timestamp TEXT PRIMARY KEY,
                cpu_percent REAL,
                memory_percent REAL,
                disk_usage REAL,
                network_io TEXT
            )
        ''')
        
        # Tabla para métricas de API
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS api_metrics (
                timestamp TEXT PRIMARY KEY,
                endpoint TEXT,
                response_time REAL,
                status_code INTEGER,
                error_message TEXT
            )
        ''')
        
        # Tabla para predicciones
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS predictions (
                timestamp TEXT PRIMARY KEY,
                input_data TEXT,
                predicted_value REAL,
                confidence_lower REAL,
                confidence_upper REAL
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def get_system_metrics(self) -> Dict[str, Any]:
        """Obtener métricas del sistema"""
        return {
            'cpu_percent': psutil.cpu_percent(interval=1),
            'memory_percent': psutil.virtual_memory().percent,
            'disk_usage': psutil.disk_usage('/').percent,
            'network_io': dict(psutil.net_io_counters()._asdict())
        }
    
    def check_api_health(self) -> Dict[str, Any]:
        """Verificar salud de la API"""
        try:
            start_time = time.time()
            response = requests.get(f"{self.api_url}/health", timeout=5)
            response_time = time.time() - start_time
            
            return {
                'status_code': response.status_code,
                'response_time': response_time,
                'is_healthy': response.status_code == 200,
                'error_message': None
            }
        except Exception as e:
            return {
                'status_code': 0,
                'response_time': 0,
                'is_healthy': False,
                'error_message': str(e)
            }
    
    def test_prediction_endpoint(self) -> Dict[str, Any]:
        """Probar endpoint de predicción"""
        test_data = {
            "age": 25,
            "goals_season": 15,
            "assists_season": 8,
            "minutes_played": 2500,
            "pass_accuracy": 0.85,
            "defensive_actions": 60,
            "league_level": 1,
            "position": "FWD"
        }
        
        try:
            start_time = time.time()
            response = requests.post(f"{self.api_url}/predict", json=test_data, timeout=10)
            response_time = time.time() - start_time
            
            result = {
                'status_code': response.status_code,
                'response_time': response_time,
                'error_message': None
            }
            
            if response.status_code == 200:
                data = response.json()
                result['predicted_value'] = data['predicted_value']
                result['confidence_interval'] = data['confidence_interval']
            
            return result
        except Exception as e:
            return {
                'status_code': 0,
                'response_time': 0,
                'error_message': str(e)
            }
    
    def store_metrics(self, system_metrics: Dict, api_health: Dict, prediction_test: Dict):
        """Almacenar métricas en base de datos"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        timestamp = datetime.now().isoformat()
        
        # Almacenar métricas del sistema
        cursor.execute('''
            INSERT OR REPLACE INTO system_metrics 
            (timestamp, cpu_percent, memory_percent, disk_usage, network_io)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            timestamp,
            system_metrics['cpu_percent'],
            system_metrics['memory_percent'],
            system_metrics['disk_usage'],
            json.dumps(system_metrics['network_io'])
        ))
        
        # Almacenar métricas de API health
        cursor.execute('''
            INSERT OR REPLACE INTO api_metrics 
            (timestamp, endpoint, response_time, status_code, error_message)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            timestamp,
            '/health',
            api_health['response_time'],
            api_health['status_code'],
            api_health['error_message']
        ))
        
        # Almacenar métricas de predicción
        cursor.execute('''
            INSERT OR REPLACE INTO api_metrics 
            (timestamp, endpoint, response_time, status_code, error_message)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            timestamp + '_predict',
            '/predict',
            prediction_test['response_time'],
            prediction_test['status_code'],
            prediction_test['error_message']
        ))
        
        conn.commit()
        conn.close()
    
    def run_monitoring_cycle(self):
        """Ejecutar un ciclo de monitoreo"""
        self.logger.info("Iniciando ciclo de monitoreo")
        
        # Obtener métricas
        system_metrics = self.get_system_metrics()
        api_health = self.check_api_health()
        prediction_test = self.test_prediction_endpoint()
        
        # Log de métricas
        self.logger.info(f"Sistema - CPU: {system_metrics['cpu_percent']:.1f}%, RAM: {system_metrics['memory_percent']:.1f}%")
        self.logger.info(f"API Health - Status: {api_health['status_code']}, Time: {api_health['response_time']:.3f}s")
        self.logger.info(f"Prediction - Status: {prediction_test['status_code']}, Time: {prediction_test['response_time']:.3f}s")
        
        # Alertas
        if system_metrics['cpu_percent'] > 80:
            self.logger.warning(f"CPU usage alto: {system_metrics['cpu_percent']:.1f}%")
        
        if system_metrics['memory_percent'] > 80:
            self.logger.warning(f"Memory usage alto: {system_metrics['memory_percent']:.1f}%")
        
        if not api_health['is_healthy']:
            self.logger.error(f"API no saludable: {api_health['error_message']}")
        
        if api_health['response_time'] > 1.0:
            self.logger.warning(f"Respuesta lenta en /health: {api_health['response_time']:.3f}s")
        
        if prediction_test['response_time'] > 2.0:
            self.logger.warning(f"Respuesta lenta en /predict: {prediction_test['response_time']:.3f}s")
        
        # Almacenar métricas
        self.store_metrics(system_metrics, api_health, prediction_test)
        
        return {
            'system': system_metrics,
            'api_health': api_health,
            'prediction_test': prediction_test
        }
    
    def start_monitoring(self, interval: int = 60):
        """Iniciar monitoreo continuo"""
        self.logger.info(f"Iniciando monitoreo continuo (intervalo: {interval}s)")
        
        try:
            while True:
                self.run_monitoring_cycle()
                time.sleep(interval)
        except KeyboardInterrupt:
            self.logger.info("Monitoreo detenido por usuario")
        except Exception as e:
            self.logger.error(f"Error en monitoreo: {e}")

if __name__ == "__main__":
    monitor = APIMonitor()
    monitor.start_monitoring(interval=30)  # Monitorear cada 30 segundos
'''

with open('monitor_api.py', 'w') as f:
    f.write(monitoring_script)

print("Sistema de monitoreo creado en 'monitor_api.py'")

## 7. Ejercicios Prácticos

### Ejercicio 1: Desplegar la API Localmente

1. **Ejecutar la API:**
   ```bash
   python api_main.py
   ```

2. **Probar la API:**
   ```bash
   python test_api.py
   ```

3. **Acceder a la documentación:**
   - Visitar: `http://localhost:8000/docs`

### Ejercicio 2: Containerizar con Docker

1. **Construir la imagen:**
   ```bash
   docker build -t player-value-api .
   ```

2. **Ejecutar el contenedor:**
   ```bash
   docker run -p 8000:8000 player-value-api
   ```

3. **Probar el contenedor:**
   ```bash
   python test_api.py
   ```

### Ejercicio 3: Implementar Monitoreo

1. **Instalar dependencias adicionales:**
   ```bash
   pip install psutil
   ```

2. **Ejecutar monitoreo:**
   ```bash
   python monitor_api.py
   ```

3. **Revisar logs y métricas:**
   - Logs en `api_monitor.log`
   - Métricas en `monitoring.db`

### Ejercicio 4: Análisis de Performance

1. **Generar carga:**
   ```bash
   # Ejecutar múltiples requests concurrentes
   for i in {1..50}; do python test_api.py & done
   ```

2. **Analizar métricas:**
   - CPU y memoria durante carga
   - Tiempo de respuesta
   - Tasa de éxito

3. **Optimizar si es necesario:**
   - Ajustar configuración de uvicorn
   - Implementar caching
   - Optimizar modelo

### Ejercicio 5: Despliegue en la Nube

1. **Preparar para Heroku:**
   - Crear `Procfile`
   - Configurar variables de entorno
   - Ajustar configuración de puerto

2. **Desplegar:**
   ```bash
   heroku create your-app-name
   git push heroku main
   ```

3. **Probar en producción:**
   - Actualizar URL en test_api.py
   - Ejecutar tests
   - Verificar logs

## Reflexiones y Tareas

### Preguntas de Reflexión

1. **¿Qué desafíos enfrentaste al desplegar el modelo?**
2. **¿Cómo asegurarías la calidad del modelo en producción?**
3. **¿Qué métricas adicionales implementarías?**
4. **¿Cómo manejarías el versionado de modelos?**
5. **¿Qué estrategias usarías para escalabilidad?**

### Tareas para Casa

1. **Implementar autenticación:** Agregar API keys o JWT tokens
2. **Crear pipeline CI/CD:** Automatizar testing y despliegue
3. **Implementar A/B testing:** Comparar versiones de modelos
4. **Agregar data drift detection:** Monitorear cambios en datos
5. **Crear dashboard de monitoreo:** Visualizar métricas en tiempo real

### Recursos Adicionales

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Docker for Python](https://docs.docker.com/language/python/)
- [MLOps Best Practices](https://ml-ops.org/)
- [Monitoring ML Systems](https://christophergs.com/machine%20learning/2020/03/14/how-to-monitor-machine-learning-models/)
- [Deploying ML Models](https://mlinproduction.com/)

---

**¡Felicitaciones!** Has completado la práctica de despliegue de modelos en producción. Has aprendido a crear APIs robustas, containerizar aplicaciones, implementar monitoreo y desplegar en la nube. Estos son skills fundamentales para cualquier científico de datos que quiera llevar sus modelos al mundo real.